<a href="https://colab.research.google.com/github/alexmrin/Image-deep-learning/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
import torch.optim as optim
import torchvision.utils as vutils
import torch.nn.functional as F
import torch.nn.init as init

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **Settings**

In [ ]:
def set_all_seeds(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

# Hyperparameters
batch_size = 64
lr = 0.1
num_epochs = 100
random_seed = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
set_all_seeds(random_seed)
if device == 'cuda':
  current_seed = torch.cuda.initial_seed()
else:
  current_seed = torch.initial_seed()

if random_seed == current_seed: print('set seeds')
device

set seeds


'cuda'

In [ ]:
!mkdir data

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


dataset = CIFAR10(root='./data', train=True, transform=train_transform, download=True)
train_dataset, val_dataset = random_split(dataset=dataset, lengths=[0.9, 0.1])
test_dataset = CIFAR10(root='./data', train=False, transform=test_transform, download=False)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

100%|██████████| 170498071/170498071 [00:04<00:00, 41965950.71it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


### Model Architecture

In [ ]:
class Resnet(nn.Module):
  def __init__(self):
    super().__init__()

    block16 = ResidualBlock(16)
    block16_d = ResidualBlock(16, downsample=True)
    block32 = ResidualBlock(32)

    block32_d = ResidualBlock(32, downsample=True)
    block64 = ResidualBlock(64)

    #input size is Bx3x32x32
    self.features = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(num_features=16),
        nn.PReLU(),

        block16,
        block16,
        block16,

        block16_d,
        block32,
        block32,

        block32_d,
        block64,
        block64,

        nn.AdaptiveAvgPool2d(1)
    )

    self.fc = nn.Linear(64, 10)
    init.xavier_uniform_(self.fc.weight)
    init.constant_(self.fc.bias, 0)

  def forward(self, x):
    features = self.features(x)
    fc = features.view(features.shape[0], -1)
    output = self.fc(fc)
    return output


# residual block
class ResidualBlock(nn.Module):
  def __init__(self, num_channels, downsample=False):
    super().__init__()

    self.downsample = downsample
    self.prelu = nn.PReLU()
    self.projection = nn.Sequential(
        nn.Conv2d(in_channels=num_channels, out_channels=num_channels*2, kernel_size=1, stride=2, padding=0),
        nn.BatchNorm2d(num_features=num_channels*2)
    )
    self.res_mapping_down = nn.Sequential(
        nn.Conv2d(in_channels=num_channels, out_channels=num_channels*2, kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(num_features=num_channels*2),
        nn.PReLU(),

        nn.Conv2d(in_channels=num_channels*2, out_channels=num_channels*2, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(num_features=num_channels*2)
    )
    self.res_mapping = nn.Sequential(
        nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(num_features=num_channels),
        nn.PReLU(),

        nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(num_features=num_channels)
    )

  def forward(self, x):
    if self.downsample:
      return self.prelu(self.res_mapping_down(x) + self.projection(x))
    else:
      return self.prelu(self.res_mapping(x) + x)


### Training Loop

In [ ]:
model = Resnet()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
  model.train()
  running_loss = 0
  for i, (images, labels) in enumerate(tqdm(train_loader)):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    predictions = model(images)
    loss = criterion(predictions, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  train_losses.append(running_loss/len(train_loader))

  print(f'Epoch: {epoch+1}/{num_epochs} | Training Loss: {running_loss/len(train_loader)}')

  model.eval()
  running_loss = 0
  with torch.no_grad():
    for i, (images, labels) in enumerate(val_loader):
      images, labels = images.to(device), labels.to(device)

      predictions = model(images)
      loss = criterion(predictions, labels)
      running_loss += loss.item()

  scheduler.step()
  val_losses.append(running_loss/len(val_loader))

  print(f'Epoch: {epoch+1}/{num_epochs} | Valid Loss: {running_loss/len(val_loader)}')

plt.title('Loss Graph')
sns.lineplot(train_losses, label='Training Loss')
sns.lineplot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

100%|██████████| 352/352 [00:31<00:00, 11.18it/s]


Epoch: 1/100 | Training Loss: 1.8189817514609208


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 1/100 | Valid Loss: 2.00007406771183


 17%|█▋        | 60/352 [00:05<00:24, 11.91it/s]


KeyboardInterrupt: ignored

In [ ]:
all_predictions = []
all_labels = []

model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Compute model predictions
        predictions = model(images)
        predictions = torch.argmax(predictions, dim=1)

        # Append predictions and labels to lists
        all_predictions.extend(predictions.detach().cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Evaluation Metrics
accuracy = accuracy_score(all_labels, all_predictions)
print(f'Accuracy on test set: {accuracy}')